<a href="https://colab.research.google.com/github/yukinaga/bert_nlp/blob/main/section_5/01_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 学術分野の分類（複数）
## jst_categorize_multi.ipynb
kakenhi_fine_tuning.ipynbでファインチューニングにより作成したモデルを用いて、新たな複数の学術文書（JSTやAMEDを含めた科研費に限らない）を11の大区分に分類するプログラム。  

### ライブラリのインストール
SageMakerでは初回だけで良い（Google colaboratoryでは毎回）  
ライブラリTransformers、およびnlpをインストールします。  

In [ ]:
# pipのアップデート（たまに走らせても良いかも）
!pip list
!python -m pip install --upgrade pip

In [ ]:
# ライブラリのインストール
!pip install torch
!pip install transformers
!pip install nlp
!pip install datasets
!pip install fugashi
!pip install ipadic

### Google ドライブとの連携  
以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。

In [ ]:
# SageMakerでは不要
# from google.colab import drive
# drive.mount("/content/drive/")

## 複数の学術文書を分類
学習済み（ファインチューニング済み）のモデルを読み込み、複数の学術文書（実際には科研費概要テキスト）を分類する

### モデルの読み込み
保存済みのモデルを読み込みます。

In [ ]:
from transformers import BertForSequenceClassification, BertJapaneseTokenizer

#data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # SageMaker

loaded_model = BertForSequenceClassification.from_pretrained(data_path) 
loaded_model.cuda() # GPU対応
loaded_tokenizer = BertJapaneseTokenizer.from_pretrained(data_path)

print("Finish")

### 分類精度検証用の科研費データ読み込み

In [ ]:
import pandas as pd

print("Start")

open_original_csv = "RUC共同研究_概要_NoJSPS.csv" # 
data_path = "../data/"

# csvファイルを開く
raw_data2 = pd.read_csv(data_path + open_original_csv) # dtype="object"必要？

# 読み込んだデータをチェック
# raw_data.info()


# 今後必要な行だけを取り出し、リネーム
kadaiDai2 = raw_data2[["ID", "FA", "description"]]
kadaiDai2['tabDai'] = 0
kadaiDai2.columns = ["ID", "FA", "Abst", "tabDai"]

# kadaiDai2 = kadaiDai2[kadaiDai2['FA'] != 'JSPS']

kadaiDai2

In [ ]:
print("Start")

# 課題番号の重複を確認。課題番号でソートする。
kadaiDai2["ID"].duplicated().any()
# kadaiDai2 = kadaiDai2.set_index("ID") # IDをインデックスに設定するコード
kadaiDai2 = kadaiDai2.sort_values("ID")

# Abstが空欄の課題を削除
print("オリジナルの課題数： %5d" % len(kadaiDai2))
print("概要が空白の課題数： %5d" % len(kadaiDai2[kadaiDai2["Abst"].isna()]))
kadaiDai2 = kadaiDai2.dropna(subset=["Abst"])
kadaiDai2("空白を除いた課題数： %5d" % len(kadaiDai2))

# Abst中の改行コードを削除
kadaiDai2 = kadaiDai2.replace('\r', '', regex=True)
kadaiDai2 = kadaiDai2.replace('\n', '', regex=True)
kadaiDai2 = kadaiDai2.replace('　', '', regex=True) # 全角スペース

# Abstが英語のみの課題を削除
num_jpen = len(kadaiDai2)
kadaiDai2 = kadaiDai2[kadaiDai2["Abst"].str.contains(r'[ぁ-んァ-ン]')]
num_jp   = len(kadaiDai2)
print("日本語＋英語： %5d" % num_jpen)
print("英語　　　　： %5d" % (num_jpen - num_jp))
print("日本語　　　： %5d" % num_jp)

print("Finish")

### 分類精度を複数ファイルで確認
３時間程度かかるかも  
sagemakerでは40分程度

In [ ]:
import glob  # ファイルの取得に使用
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

results_binary = 'results'
# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # sagemaker
Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]


max_length = 512

num_data = len(kadaiDai2)
print("分類する課題数： %d" % num_data)
# num_data = 100
num_category = len(Daikubun)
# results = torch.zeros(num_data, num_category) # テンソルで変数を用意
results = np.zeros((num_data, num_category)) # メモリーが足りないと言われるのでテンソルではなくnumpy arrayにしてみた

for m in tqdm(range(num_data)):
  words = loaded_tokenizer.tokenize(kadaiDai2["Abst"][m])
  word_ids = loaded_tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
  word_tensor = torch.tensor([word_ids[:max_length]])  # テンソルに変換
  
  # y = loaded_model(word_tensor) # GPU未対応時の予測
  y = loaded_model(word_tensor.cuda())  # GPU対応時の予測

  # results[m,:] = y[0]
  results[m,:] = y[0].cpu().detach().numpy()  # テンソルをCPUにコピーしてからNumPy配列に変換
  # results[m,:] = y[0].detach().numpy() # テンソルをnumpy arrayに変換

# 変数をとりあえずバイナリで保存
np.save(data_path+results_binary, results) # 計算結果をとりあえずバイナリで保存

print(results.shape)
results

### ここで一度保存
時間がかかる処理のあとなので、この段階で生成できたデータをファイルに保存

In [ ]:
# コードが不十分
data_path = "./" # sagemaker
Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]

results = np.load(data_path+results_binary+".npy")

# 変数をとりあえずバイナリで保存
#np.save(data_path+results_binary, results) # 計算結果をとりあえずバイナリで保存
np.savez(data_path+"categolization_results", results, kadaiDai2, Daikubun)

## 分類結果を表示

### 複数ファイル

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

show_num = 0

results_binary = "results"
# data_path = "/content/drive/My Drive/bert_nlp/section_5/" # Google colaboratory
data_path = "./" # sagemaker
Daikubun = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]

# num_category = len(Daikubun)

results_2 = np.load(data_path+results_binary+".npy")



#results_3 = np.load(data_path+"categolization_results.npz", allow_pickle=True)
#results_2 = results_3['arr_0']
#kadaiDai2 = results_3['arr_1']
#Daikubun  = results_3['arr_2']

print("ロードした推定確率データのサイズ %d %d" % results_2.shape)

num_category = len(Daikubun)

results_2 = torch.tensor(results_2) # Softmax関数を使うためにテンソルに変換
m = torch.nn.Softmax(dim=1) # Softmax関数で確率に変換
results_2 = m(results_2)
results_2 = results_2.numpy() # numpy arrayに戻す


# 保存
df = pd.DataFrame(results_2)
df.to_csv('output.csv', encoding='shift-jis')
df